In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
import numpy as np
from scipy.spatial.distance import cosine


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\justi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
!pip install transformers

In [3]:
df = pd.read_csv('data/billboard-lyrics-spotify.csv', sep=',', header=0)


In [6]:
df['year'] = pd.to_datetime(df['year'], format='%Y')

In [8]:
df['decade'] = df['year'].dt.year // 10 * 10

In [11]:
grouped_df = df.groupby('decade')

In [14]:
result = grouped_df['lyrics'].count()
result

decade
1960    975
1970    986
1980    994
1990    933
2000    919
2010    709
Name: lyrics, dtype: int64

In [19]:
grouped_df.get_group(2010)

,artist_all,artist_base,rank,song,year,artist_featured,song_clean,artist_clean,lyrics,acousticness,...,speechiness,tempo,time_signature,valence,duration_min,num_words,words_per_sec,num_uniq_words,decade,uniq_ratio
4850,kesha,kesha,1,tik tok,2010-01-01,NaN,tik tok,kesha,grab my glasses im out the door im gonna hit ...,0.07460,...,0.1160,120.032,4.0,0.735,3.328217,458.0,2.293521,136.0,2010,3.367647
4851,lady antebellum,lady antebellum,2,need you now,2010-01-01,NaN,need you now,lady antebellum,picture perfect memories scattered all around...,0.09270,...,0.0303,107.943,4.0,0.231,4.626217,215.0,0.774571,77.0,2010,2.792208
4852,train,train,3,"hey, soul sister",2010-01-01,NaN,hey soul sister,train,heyy heeey heey your lipstick stains on the f...,0.21700,...,0.0436,97.030,4.0,0.768,3.611117,284.0,1.310767,115.0,2010,2.469565
4853,katy perry featuring snoop dogg,katy perry,4,california gurls,2010-01-01,snoop dogg,california gurls,katy perry,greetings loved ones lets take a journey i kn...,0.00446,...,0.0569,125.014,4.0,0.425,3.910883,445.0,1.896417,162.0,2010,2.746914
4854,usher featuring will.i.am,usher,5,omg,2010-01-01,will.i.am,omg,usher,oh my gosh baby let me i did it again so im g...,0.19800,...,0.0332,129.998,4.0,0.326,4.491550,526.0,1.951813,105.0,2010,5.009524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5561,camila cabello featuring young thug,camila cabello,96,havana,2017-01-01,young thug,havana,camila cabello,hey half of my heart is in havana ooh na na h...,0.18400,...,0.0300,104.988,4.0,0.394,3.621783,350.0,1.610625,102.0,2010,3.431373
5562,maroon 5 featuring sza,maroon 5,97,what lovers do,2017-01-01,sza,what lovers do,maroon 5,say say say hey hey now baby oh mama dont pla...,0.08050,...,0.0693,109.959,4.0,0.420,3.330817,366.0,1.831383,75.0,2010,4.880000
5563,blackbear,blackbear,98,do re mi,2017-01-01,NaN,do re mi,blackbear,do re mi fa so yeah yeah yeah oh do re mi fa ...,0.00419,...,0.0465,110.977,3.0,0.154,3.533783,382.0,1.801657,122.0,2010,3.131148
5564,xxxtentacion,xxxtentacion,99,look at me!,2017-01-01,NaN,look at me,xxxtentacion,ayy im like bitch who is your mans ayy cant k...,0.25900,...,0.2820,139.059,4.0,0.349,2.105767,337.0,2.667279,106.0,2010,3.179245


In [16]:
grouped_df.groups.keys()

dict_keys([1960, 1970, 1980, 1990, 2000, 2010])

In [4]:
df[df['song'] == 'do re mi']['lyrics'].values[0]

' do re mi fa so yeah yeah yeah oh do re mi fa so yeah yeah yeah yeah if i could go back to the day we met i probably would just stay in bed you run your mouth all over town and this one goes out to the sound of breaking glass on my range rover pay me back or bitch its over all the presents i would send fuck my friends behind my shoulder next time ima stay asleep i pray the lord my soul to keep oh and you got me thinking lately bitch you crazy and nothings ever good enough i wrote a little song for ya it go like do re mi fa so fuckin done with you girl so fuckin done with all the games you play i aint no tic tac toe send the x and os on another note im do re mi fa so fuckin done with you baby so send the x and os on another note im ghost if i could go back to the day we met i probably wouldve stayed in bed you wake up everyday and make me feel like im incompetent designer shoes and xanax tabs compliments your make up bag you never had to buy yourself a drink cause everybody want to tap

In [5]:
# df['lyrics']
lyrics = [item for item in df['lyrics']]
lyrics_strings = list(map(str, lyrics))
lyrics_words = [word_tokenize(item) for item in lyrics_strings]

In [6]:
word_counts = [len(item) for item in lyrics_words]
avg_word_counts = np.mean(word_counts)
min_words = np.min(word_counts)
max_words = np.max(word_counts)
print('Average number of words per song: {}'.format(avg_word_counts))
print('Minimum number of words per song: {}'.format(min_words))
print('Maximum number of words per song: {}'.format(max_words))

Average number of words per song: 303.3695652173913
Minimum number of words per song: 0
Maximum number of words per song: 1153


In [7]:
sw = set(stopwords.words('english'))
lowercased = [[word.lower() for word in item] for item in lyrics_words]
removed_punctuation = [[word for word in review if word.isalnum()] for review in lowercased]
removed_stopwords = [[word for word in review if word not in sw] for review in removed_punctuation]

In [8]:
ps = PorterStemmer()
stemmed = [[ps.stem(word) for word in review] for review in removed_stopwords]


In [ ]:
from nltk.util import ngrams

unigrams = [list(ngrams(item, 1)) for item in removed_punctuation]
bigrams = [list(ngrams(item, 2)) for item in removed_punctuation]
trigrams = [list(ngrams(item, 3)) for item in removed_punctuation]

In [39]:
# import BERT model
import torch
from transformers import BertTokenizer, BertModel

# load tokenizer (same one trained with the model)
tokenizer = BertTokenizer.from_pretrained("unitary/toxic-bert")

# load pre-trained BERT model weights
model = BertModel.from_pretrained(
    "unitary/toxic-bert",
    output_hidden_states = True, # Whether the model returns all hidden-states.
)
# model = torch.hub.load('unitaryai/detoxify','multilingual_toxic_xlm_r')
# model.eval()



In [48]:
e1 = df[df['song'] == 'do re mi']['lyrics'].values[0]
input1 = torch.tensor([tokenizer.encode(e1, add_special_tokens=True)])
input1

tensor([[  101,  2079,  2128,  2771,  6904,  2061,  3398,  3398,  3398,  2821,
          2079,  2128,  2771,  6904,  2061,  3398,  3398,  3398,  3398,  2065,
          1045,  2071,  2175,  2067,  2000,  1996,  2154,  2057,  2777,  1045,
          2763,  2052,  2074,  2994,  1999,  2793,  2017,  2448,  2115,  2677,
          2035,  2058,  2237,  1998,  2023,  2028,  3632,  2041,  2000,  1996,
          2614,  1997,  4911,  3221,  2006,  2026,  2846, 13631,  3477,  2033,
          2067,  2030,  7743,  2049,  2058,  2035,  1996,  7534,  1045,  2052,
          4604,  6616,  2026,  2814,  2369,  2026,  3244,  2279,  2051, 10047,
          2050,  2994,  6680,  1045, 11839,  1996,  2935,  2026,  3969,  2000,
          2562,  2821,  1998,  2017,  2288,  2033,  3241,  9906,  7743,  2017,
          4689,  1998,  2498,  2015,  2412,  2204,  2438,  1045,  2626,  1037,
          2210,  2299,  2005,  8038,  2009,  2175,  2066,  2079,  2128,  2771,
          6904,  2061,  6616,  2378,  2589,  2007,  

In [1]:
for i, token_str in enumerate(tokenizer.tokenize(e1)):
    print(i, token_str)
print()

NameError: name 'tokenizer' is not defined

In [53]:
with torch.no_grad():
    output = model(input1)
    hidden1 = output[2]

In [55]:
hidden = [hidden1]
for i in range(len(hidden)):
    print(f'Sentence {i}')
    print ("Number of layers:", len(hidden_i), "  (initial embeddings + 12 BERT layers)")
    print ("Number of batches:", len(hidden_i[0]))
    print ("Number of tokens:", len(hidden_i[0][0]))
    print ("Number of hidden units:", len(hidden_i[0][0][0]))
    print()

Sentence 0
Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 413
Number of hidden units: 768



In [56]:
token_embeddings = []

for i in range(len(hidden)):
    hidden_i = hidden[i]
    token_embeddings_i = torch.stack(hidden_i, dim=0)
    token_embeddings_i = torch.squeeze(token_embeddings_i, dim=1)
    token_embeddings_i = token_embeddings_i.permute(1,0,2)
    print(token_embeddings_i.size())
    token_embeddings.append(token_embeddings_i)

torch.Size([413, 13, 768])


In [58]:
idx1 = 7

token_vecs_cat = []
for i, doc_rep in enumerate(token_embeddings):
    token_Vec = []
    for token_rep in doc_rep:
        cat_vec = torch.cat((token_rep[-1], token_rep[-2], token_rep[-3], token_rep[-4]), dim=0)
        token_Vec.append(cat_vec)
    token_vecs_cat.append(token_Vec)

print('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

words = []
words.append(token_vecs_cat[0][idx1])

Shape is: 1 x 413


In [ ]:

sentence_embedding = []

for i, doc in enumerate(token_vecs_cat):
    sentence_embedding_i = torch.mean(torch.stack(doc), dim=0)
    sentence_embedding.append(sentence_embedding_i)

cos_s1_s2 = 1 - cosine(sentence_embedding[0], sentence_embedding[1])

print("s1: ", e1)
print()
print('Vector similarity for ')